In [1]:
import pandas as pd
import numpy as np
import os
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
audio_dataset_path="genres_original"
metadata=pd.read_csv("features_30_sec.csv")
metadata.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [3]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [5]:
# pip install tqdm

In [6]:
from tqdm import tqdm
### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients

extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    try:
        final_class_labels=row["label"]
        file_name = os.path.join(os.path.abspath(audio_dataset_path), final_class_labels+'/',str(row["filename"]))    
        data=features_extractor(file_name)
        extracted_features.append([data,final_class_labels])
    except Exception as e:
#         print(f"Error: {e}")
        continue

553it [01:04, 10.86it/s]C:\Users\kaushal prajapat\anaconda3\envs\tf2.5\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
1000it [01:40,  9.91it/s]


In [7]:
### converting extracted_features to Pandas dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-113.59882, 121.57067, -19.162262, 42.36394, ...",blues
1,"[-207.52383, 123.98514, 8.94702, 35.86715, 2.9...",blues
2,"[-90.757164, 140.44087, -29.084547, 31.686693,...",blues
3,"[-199.57513, 150.0861, 5.663404, 26.855278, 1....",blues
4,"[-160.35417, 126.20948, -35.581394, 22.139256,...",blues


In [8]:
extracted_features_df['class'].value_counts()

blues        100
classical    100
country      100
disco        100
hiphop       100
metal        100
pop          100
reggae       100
jazz          99
rock          54
Name: class, dtype: int64

In [9]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [10]:
X.shape

(953, 40)

In [11]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [12]:
y.shape

(953, 10)

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.33,random_state=0)

In [14]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(638, 40)
(315, 40)
(638, 10)
(315, 10)


In [15]:
import tensorflow as tf

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [17]:
### No of classes
num_labels=y.shape[1]

In [18]:
y.shape[1]

10

In [19]:
model=Sequential()
model.add(Dense(1024,input_shape=(40,), activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.3))

###final layer
model.add(Dense(num_labels, activation="softmax"))

In [20]:
# model.summary()

In [21]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [22]:
import time
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)

In [23]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 250
num_batch_size = 8

checkpointer = ModelCheckpoint(filepath=f'saved_models/audio_classification_{current_time}.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

history = model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/250
80/80 [==============================] - 5s 22ms/step - loss: 4.7779 - accuracy: 0.0956 - val_loss: 2.3015 - val_accuracy: 0.1333
Epoch 2/250
80/80 [==============================] - 1s 10ms/step - loss: 2.5291 - accuracy: 0.1270 - val_loss: 2.2372 - val_accuracy: 0.1746
Epoch 3/250
80/80 [==============================] - 1s 9ms/step - loss: 2.3810 - accuracy: 0.1520 - val_loss: 2.1954 - val_accuracy: 0.2254
Epoch 4/250
80/80 [==============================] - 1s 9ms/step - loss: 2.3093 - accuracy: 0.1677 - val_loss: 2.2134 - val_accuracy: 0.2032
Epoch 5/250
80/80 [==============================] - 1s 9ms/step - loss: 2.2414 - accuracy: 0.1614 - val_loss: 2.2032 - val_accuracy: 0.1810
Epoch 6/250
80/80 [==============================] - 1s 9ms/step - loss: 2.1977 - accuracy: 0.1944 - val_loss: 2.0816 - val_accuracy: 0.2095
Epoch 7/250
80/80 [==============================] - 1s 9ms/step - loss: 2.1616 - accuracy: 0.1928 - val_loss: 2.0820 - val_accuracy: 0.2730
Epoch 8/250

80/80 [==============================] - 1s 13ms/step - loss: 1.2875 - accuracy: 0.5580 - val_loss: 2.0550 - val_accuracy: 0.4762
Epoch 59/250
80/80 [==============================] - 1s 16ms/step - loss: 1.2040 - accuracy: 0.5627 - val_loss: 2.0515 - val_accuracy: 0.4984
Epoch 60/250
80/80 [==============================] - 1s 17ms/step - loss: 1.2317 - accuracy: 0.5721 - val_loss: 2.0053 - val_accuracy: 0.4825
Epoch 61/250
80/80 [==============================] - 1s 17ms/step - loss: 1.1467 - accuracy: 0.5674 - val_loss: 2.0070 - val_accuracy: 0.4984
Epoch 62/250
80/80 [==============================] - 1s 18ms/step - loss: 1.1813 - accuracy: 0.5752 - val_loss: 1.9612 - val_accuracy: 0.5238
Epoch 63/250
80/80 [==============================] - 1s 13ms/step - loss: 1.1376 - accuracy: 0.5940 - val_loss: 2.0682 - val_accuracy: 0.5270
Epoch 64/250
80/80 [==============================] - 1s 10ms/step - loss: 1.1862 - accuracy: 0.6097 - val_loss: 2.2865 - val_accuracy: 0.5524
Epoch 65/250

80/80 [==============================] - 1s 15ms/step - loss: 0.5568 - accuracy: 0.8213 - val_loss: 2.7315 - val_accuracy: 0.6127
Epoch 116/250
80/80 [==============================] - 1s 15ms/step - loss: 0.7079 - accuracy: 0.7962 - val_loss: 2.5315 - val_accuracy: 0.5937
Epoch 117/250
80/80 [==============================] - 1s 12ms/step - loss: 0.6947 - accuracy: 0.7900 - val_loss: 2.0984 - val_accuracy: 0.6095
Epoch 118/250
80/80 [==============================] - 1s 10ms/step - loss: 0.6442 - accuracy: 0.8009 - val_loss: 2.2879 - val_accuracy: 0.5905
Epoch 119/250
80/80 [==============================] - 1s 9ms/step - loss: 0.7312 - accuracy: 0.8119 - val_loss: 2.4075 - val_accuracy: 0.6000
Epoch 120/250
80/80 [==============================] - 1s 12ms/step - loss: 0.6483 - accuracy: 0.7884 - val_loss: 2.5649 - val_accuracy: 0.5651
Epoch 121/250
80/80 [==============================] - 1s 15ms/step - loss: 0.6613 - accuracy: 0.7947 - val_loss: 2.5516 - val_accuracy: 0.5683
Epoch 1

Epoch 172/250
80/80 [==============================] - 1s 16ms/step - loss: 0.3464 - accuracy: 0.9060 - val_loss: 3.5732 - val_accuracy: 0.6286
Epoch 173/250
80/80 [==============================] - 1s 16ms/step - loss: 0.5167 - accuracy: 0.8918 - val_loss: 3.0192 - val_accuracy: 0.6063
Epoch 174/250
80/80 [==============================] - 1s 16ms/step - loss: 0.3874 - accuracy: 0.8856 - val_loss: 3.2952 - val_accuracy: 0.6222
Epoch 175/250
80/80 [==============================] - 1s 15ms/step - loss: 0.3760 - accuracy: 0.8840 - val_loss: 3.2654 - val_accuracy: 0.5937
Epoch 176/250
80/80 [==============================] - 1s 16ms/step - loss: 0.4722 - accuracy: 0.8746 - val_loss: 2.6346 - val_accuracy: 0.6381
Epoch 177/250
80/80 [==============================] - 1s 17ms/step - loss: 0.4408 - accuracy: 0.8809 - val_loss: 3.1034 - val_accuracy: 0.5905
Epoch 178/250
80/80 [==============================] - 1s 18ms/step - loss: 0.4160 - accuracy: 0.8840 - val_loss: 3.6532 - val_accuracy:

Epoch 229/250
80/80 [==============================] - 1s 15ms/step - loss: 0.3423 - accuracy: 0.9122 - val_loss: 3.7504 - val_accuracy: 0.6317
Epoch 230/250
80/80 [==============================] - 1s 15ms/step - loss: 0.2467 - accuracy: 0.9263 - val_loss: 4.2622 - val_accuracy: 0.6508
Epoch 231/250
80/80 [==============================] - 1s 10ms/step - loss: 0.2686 - accuracy: 0.9185 - val_loss: 3.6665 - val_accuracy: 0.6381
Epoch 232/250
80/80 [==============================] - 1s 9ms/step - loss: 0.3448 - accuracy: 0.9185 - val_loss: 3.7465 - val_accuracy: 0.6317
Epoch 233/250
80/80 [==============================] - 1s 9ms/step - loss: 0.3500 - accuracy: 0.9169 - val_loss: 3.7429 - val_accuracy: 0.6381
Epoch 234/250
80/80 [==============================] - 1s 15ms/step - loss: 0.2675 - accuracy: 0.9279 - val_loss: 3.7828 - val_accuracy: 0.6127
Epoch 235/250
80/80 [==============================] - 1s 15ms/step - loss: 0.2776 - accuracy: 0.9232 - val_loss: 4.6205 - val_accuracy: 0

In [24]:
model.evaluate(X_test,y_test,verbose=0)

[3.2782464027404785, 0.5968254208564758]

In [31]:
filename="test_data/p.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)

predict_x = model.predict(mfccs_scaled_features)
predicted_label=np.argmax(predict_x,axis=1)
print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
prediction_class

1/1 [==============================] - 0s 31ms/step
[7]


array(['pop'], dtype='<U9')

In [ ]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [35]:
import pickle